In [1]:
import pandas as pd
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
import csv
from datetime import datetime

In [10]:
# Create a dictionary that maps the run# to host_density { run#: host_density }

# paramfile = "/media/hill/DATA-LINUX/abm-data/host-density/testparams2hab5"
paramfile = "/media/hill/DATA-LINUX/abm-data/host-density/params-Sept2-hab05"
paramd = {}
with open(paramfile, 'r') as file:
    for line in file:
        result = line.replace("\t",",").replace('\n','').split(',')

        # attempt to add the density value to run, if it doesn't exist, create it
        try:
            paramd[int(result[0]) - 1].append((float(result[4])))
        except KeyError:
            paramd[int(result[0]) - 1] = (float(result[4]))  # { run#: host_density }

for k, v in paramd.items():
    print(k,v)

0 0.1
1 0.1
2 0.1
3 0.1
4 0.1
5 0.1
6 0.1
7 0.1
8 0.1
9 0.1
10 0.3
11 0.3
12 0.3
13 0.3
14 0.3
15 0.3
16 0.3
17 0.3
18 0.3
19 0.3
20 0.5
21 0.5
22 0.5
23 0.5
24 0.5
25 0.5
26 0.5
27 0.5
28 0.5
29 0.5
30 0.7
31 0.7
32 0.7
33 0.7
34 0.7
35 0.7
36 0.7
37 0.7
38 0.7
39 0.7
40 0.9
41 0.9
42 0.9
43 0.9
44 0.9
45 0.9
46 0.9
47 0.9
48 0.9
49 0.9


In [25]:
# NEW CODE - dont build so many data structures
# step 2: read csv, create df

colnames = ['run', 'tick', 'lifestate', 'total_ixodes']
csv_file = "/media/hill/DATA-LINUX/abm-data/host-density/host-density.2020.Sep.02.hab05"
before = datetime.now()
df = pd.read_csv(csv_file, names=colnames, header=None)
after = datetime.now()

print("Run time: ", after-before)
# Reading csv takes ~20s
#





/home/hill/anaconda3/envs/abm/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (0,1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Run time:  0:00:18.998129


In [ ]:
n_ticks = df.groupby(['run'], as_index=False)
# # n_ticks = n_ticks.agg(total_ixodes=pd.NamedAgg(column='name', aggfunc='nunique'))
n_ticks = n_ticks.agg({'total_ixodes': 'nunique'})
# takes ~14s

In [19]:
n_ticks.head()

,run,total_ixodes,host_density
0,1,26984,0.1
1,2,39789,0.1
2,3,4111,0.1
3,4,15009,0.1
4,5,10619,0.1


In [20]:
# step 3
# Iterate through param dictionary and add the host density value to the associated run
for key, value in paramd.items():
    n_ticks.loc[n_ticks['run']==key, 'host_density' ] = float(value) #FIXME off by 1 with new method?
n_ticks.head()


,run,total_ixodes,host_density
0,1,26984,0.1
1,2,39789,0.1
2,3,4111,0.1
3,4,15009,0.1
4,5,10619,0.1


In [23]:
newdf = n_ticks.groupby('host_density')['total_ixodes'].agg({'mean', 'std'})
newdf.head()


,std,mean
host_density,,
0.1,NaN,23294.888889
0.3,NaN,23173.200000
0.5,NaN,16029.600000
0.7,NaN,14612.100000
0.9,NaN,22640.700000


In [24]:
# step 4 make the final df
n_ticks = n_ticks.groupby('host_density', )
n_ticks = n_ticks.agg({'total_ixodes': 'mean'})
n_ticks.head()

,total_ixodes
host_density,
0.1,23294.888889
0.3,23173.200000
0.5,16029.600000
0.7,14612.100000
0.9,22640.700000


In [114]:
mean = n_ticks.groupby('host_density')['total_ixodes'].mean()
std = n_ticks.groupby('host_density')['total_ixodes'].std()
mean

host_density
0.01    11.0
0.03    13.5
0.05    16.0
Name: total_ixodes, dtype: float64

In [61]:
# -- OLD way
# Create a dictionary that maps the run# to total ixode { run#: cumulative_ixode }
ixode_count_dict = {}
colnames = ['run', 'tick', 'lifestate', 'name']
csv_file = "/media/hill/DATA-LINUX/abm-data/host-density/testdf1hab"
df = pd.read_csv(csv_file, names=colnames, header=None, error_bad_lines=False)
for run in df.groupby('run'):
    current_df = run[1]
    ixode_count_dict[run[0]] = len(current_df['name'].unique())


In [97]:
# Creating a data frame from the cumulative_ixodes dict generated above
df_final = pd.DataFrame(ixode_count_dict.items(), columns=['run', 'total_ixodes'])
df_final['host_density'] = 0
df_final.dtypes

NameError: name 'ixode_count_dict' is not defined

In [100]:
# Iterate through param dictionary and add the host density value to the associated run
for key, value in paramd.items():
    for i in range(len(paramd)):
        # print(type(value))
        df_final.loc[df_final['run']==key, 'host_density'] = value

df_final.dtypes

NameError: name 'df_final' is not defined

In [58]:
# Now we have a df with | 'run' | 'total_ixode' | 'host_density' |
# Create a dictionary with { host_density: agg_ixodes }
agg_ixodes_dict = {}
for density in df_final.groupby('host_density'):
    tmp_df = density[1] # density is a tuple, so density[1] is the df we want
    # print(density[0])
    agg_ixodes_dict[density[0]] = tmp_df['total_ixodes'].agg('mean')


# in test file - should get 1880.75 for 0.4, 7898.5 for 0.7, and 24917 for 1.0


0.7


In [52]:
# Create a dataframe from the agg_ixodes_dict that we can then plot
df_agg_final = pd.DataFrame(agg_ixodes_dict.items(), columns=['host_density', 'agg_ixodes'])
df_agg_final.head()

,host_density,agg_ixodes
0,0.7,13.8


In [9]:
matplotlib.use('Qt5Agg')
plt.plot(df_agg_final['host_density'], df_agg_final['agg_ixodes'])
plt.show(block=True)

In [33]:
with open(paramfile, 'r') as file:
    reader = csv.reader(file)
    param_str = {rows[0]: "Lifestage: {}\nStarting Ixodes: {}\nHabitat Suitability: {}".format(
        rows[1], rows[2], rows[3]) for rows in reader}
    print(param_str)

{'1\ttick_lifestage\tadult': 'Lifestage: large_host_density\t0.7\nStarting Ixodes: tick_count\t10\nHabitat Suitability: habitat_suitability\t0.01', '2\ttick_lifestage\tadult': 'Lifestage: large_host_density\t0.7\nStarting Ixodes: tick_count\t10\nHabitat Suitability: habitat_suitability\t0.03', '3\ttick_lifestage\tadult': 'Lifestage: large_host_density\t0.7\nStarting Ixodes: tick_count\t10\nHabitat Suitability: habitat_suitability\t0.05', '4\ttick_lifestage\tadult': 'Lifestage: large_host_density\t0.7\nStarting Ixodes: tick_count\t10\nHabitat Suitability: habitat_suitability\t0.07', '5\ttick_lifestage\tadult': 'Lifestage: large_host_density\t0.7\nStarting Ixodes: tick_count\t10\nHabitat Suitability: habitat_suitability\t0.1'}


In [40]:
fig, ax = plt.subplots()
ax.set_title("Test Agg Host Density")
ax.set_ylabel("Total Ixodes")
ax.set_xlabel("Host Density")
props = dict(facecolor='wheat', alpha=0.5)
ax.plot(df_agg_final['host_density'], df_agg_final['agg_ixodes'])
plt.figtext(0.5,0.5, 'Lifestage: adult\nStarting Ixodes: 10\nHabitat Suitability: 0.05', bbox=props)
plt.show(block=True)

In [49]:
df2 = df_agg_final.copy()
ax=df2.plot()
df_agg_final.plot(ax=ax)
plt.show(block=True)

